In [2]:
# open Dart APi 주소 = https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019001

from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
from html_table_parser import parser_functions as parser
import webbrowser
import requests #웹상 정보 요청

import pandas as pd
import numpy as np
import csv
import re # 정규표현식

from zipfile import ZipFile
import xml.etree.ElementTree

from io import BytesIO  # Bytes 데이터 변환 라이브러리 

In [ ]:
# 로컬 경로 지정 
path = # insert your personal path 

# 크롬 본인인증 키 
user_agent = # insert your User Agent

# DART API Key
dart_auth = # insert your DART Authorization

## corp_code 데이터 확보

In [ ]:
with urllib.request.urlopen(url) as zipresponse:
    with ZipFile(BytesIO(zipresponse.read())) as zipfile:
        zipfile.extractall('corpCode')

In [ ]:
tree = xml.etree.ElementTree.parse('CORPCODE.xml')
root = tree.getroot()

file = open("corpcode.csv", mode = 'w')
writer = csv.writer(file)
writer.writerow(['corp_code', 'corp_name','stock_code', 'stock_name', 'modify_date'])

In [ ]:
for country in root.iter("list"):
    
    stock_code = country.findtext("stock_code")
    stock_code = stock_code.strip()
    if stock_code:
        country_dic = {
            "corp_code": country.findtext("corp_code"),
            "corp_name": country.findtext("corp_name"),
            "stock_code": country.findtext("stock_code"),
            "stock_name": country.findtext("stock_name"),
            "modify_date": country.findtext("modify_date")
            
        }
        writer.writerow(list(country_dic.values()))
# 작업 완료 후 작업중인 폴더 내에서 파일 수정후 load필요
# 본인은 corpcode.csv를 DART_corp_info.csv로 수정한 후 파일 가져옴

## rcept_no 확보

In [ ]:
DART_corp_info = pd.read_csv("./DART_corp_info.csv")

In [ ]:
# 사용 칼럼 정리
DART_corp_info = DART_corp_info[['corp_code', 'corp_code', 'corp_name_eng', 'stock_name', 'stock_code', 'ceo_nm', 'corp_cls', 'jurir_no', 'bizr_no', 'adres', 'hm_url', 'phn_no', 'fax_no', 'induty_code', 'est_dt']]
DART_corp_info.columns = ['new_corp_code', 'corp_code', 'corp_name_eng', 'stock_name', 'stock_code', 'ceo_nm', 'corp_cls', 'jurir_no', 'bizr_no', 'adres', 'hm_url', 'phn_no', 'fax_no', 'induty_code', 'est_dt']

In [ ]:
# open dart api 최신화에 따른 corpcode 데이터 수정 6자리 -> 8자리
for i in range(len(DART_corp_info['new_corp_code'])):
    DART_corp_info['new_corp_code'] = DART_corp_info['new_corp_code'].astype(str)
    
    if len(DART_corp_info['new_corp_code'][i]) == 6:
        DART_corp_info['new_corp_code'][i] = '00' + DART_corp_info['new_corp_code'][i]
    
    elif len(DART_corp_info['new_corp_code'][i]) == 7:
        DART_corp_info['new_corp_code'][i] = '0' + DART_corp_info['new_corp_code'][i]
    
    elif len(DART_corp_info['new_corp_code'][i]) == 8:
        DART_corp_info['new_corp_code'][i] = DART_corp_info['new_corp_code'][i]

In [ ]:
revised_corp_info = DART_corp_info[DART_corp_info['corp_cls'] != 'E'] #기타 상장사가 아닌 회사만 선택
revised_corp_info_new_corp_code = np.array(revised_corp_info['new_corp_code'].tolist())

In [ ]:
rcept_no_list=[]
corp_code_list =[]

for i in range(len(revised_corp_info_new_corp_code)):
    try:
        # 2020년 연결 재무제표 기준으로 데이터 확보 
        url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.xml?crtfc_key={}&corp_code={}&bsns_year=2020&reprt_code=11011&fs_div=OFS'.format(dart_auth, revised_corp_info_new_corp_code[i])
        resp = requests.get(url, headers={"user-agent": user_agent}) #정보 획득 가능 여부 확인 
        webpage = resp.content.decode('UTF-8')
        
        rcept_no =  re.findall(r'<rcept_no>(.*?)</rcept_no>', webpage)
        rcept_no = rcept_no[0]
        rcept_no_list.append(rcept_no)
        
        corp_code =  re.findall(r'<corp_code>(.*?)</corp_code>', webpage)
        corp_code = corp_code[0]
        corp_code_list.append(corp_code)

    except Exception as e:
        print(revised_corp_info_new_corp_code[i]+" is error")
        print(e)
        
corp_code_list

In [ ]:
rcp_corpcode = pd.DataFrame(zip(rcept_no_list, corp_code_list))

In [ ]:
rcp_corpcode.columns = ['rcept_no', 'new_corp_code']
rcp_corpcode.to_csv('./rcp_corpcode.csv')

## 기업별 rcept_no에 따른 dcm_no 확보

In [ ]:
rcp_corpcode = pd.read_csv('rcp_corpcode.csv')

In [ ]:
import time
import random 

# 사업보고서 다운로드 창 URL
# disclosure_viewer_url = 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo='

dcm_no_list = []
no_list = []

for i in range(len(rcp_corpcode['rcept_no'])):
    try:
        url = 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp_corpcode['rcept_no'][i])
        resp = requests.get(url, headers={"user-agent": user_agent}) #정보 획득 가능 여부 확인 
        time.sleep( random.uniform(1,3) )
        webpage = resp.content.decode('UTF-8')
        dcm_no = re.findall(r"{}', '(.*?)',".format(rcp_corpcode['rcept_no'][i]), webpage)[0]
        dcm_no_list.append(dcm_no)
        no_list.append(i)
        
    except Exception as e:
        print(e)

In [ ]:
# 데이터 프레임화
dcm_no_list_df = pd.DataFrame(dcm_no_list)

# 작업시간증가에 따른 작업 분할 및 데이터 저장 
rcp_dcm_corpcode = pd.concat([rcp_corpcode, dcm_no_list_df], axis = 1)
rcp_dcm_corpcode.columns = ['Unnamed: 0	', 'rcept_no', 'new_corp_code', 'dcm_no']
rcp_dcm_corpcode.to_csv('./rcp_dcm_corpcode.csv')

In [ ]:
# 데이터 로드 
rcp_dcm_corpcode = pd.read_csv('rcp_dcm_corpcode.csv')

## 재무 데이터 처리를 위한 데이터 pivot

In [ ]:
import time
import random 

# sheet_names = ['연결 재무상태표', '연결 포괄손익계산서', '연결 현금흐름표']

# data_total = pd.DataFrame()

array = np.arange(0,3000,1)
data_total = pd.DataFrame(array)
data_total.columns = ['index']

for i in range(len(rcp_dcm_corpcode['dcm_no'])):
    try:
        url = 'https://dart.fss.or.kr/pdf/download/excel.do?rcp_no={}&dcm_no={}&lang=ko'.format(rcp_dcm_corpcode['rcept_no'][i], rcp_dcm_corpcode['dcm_no'][i])
        time.sleep( random.uniform(1,3) )
        resp = requests.get(url, headers={"user-agent": user_agent}) #정보 획득 가능 여부 확인 
        table = BytesIO(resp.content)
        
        
        data1 = pd.read_excel(table, sheet_name = '재무상태표', skiprows=5)
        data1 = data1.iloc[:,[0,1]]
        data2 = pd.read_excel(table, sheet_name = '포괄손익계산서', skiprows=5)
        data2 = data2.iloc[:,[0,1]]
        data3 = pd.read_excel(table, sheet_name = '현금흐름표', skiprows=5)
        data3 = data3.iloc[:,[0,1]]

        data_concat = pd.concat([data1, data2, data3], axis = 0)
        
        data4 = pd.DataFrame( np.arange(len(data_concat),3000,1) )
        data4.columns = ['Unnamed: 1']
        data_concat = pd.concat([data_concat, data4], axis = 0)
        data_concat.rename(columns={data_concat.columns[1]: rcp_dcm_corpcode['dcm_no'][i] }, inplace=True)
        data_concat['index'] = np.arange(0,3000,1)

        
        data_total = pd.merge(data_total, data_concat, how = "outer", on = 'index')
        
        
    except Exception as e:
        print(e)


In [ ]:
data_total.to_csv('./data_total_재무상태표ver_notE.csv', encoding = 'euc-kr')